In [2]:
import sympy as sp
import numpy as np
from scipy import linalg
from sympy import *
from IPython.display import Math, Latex

## List of tuning parameters for position and velocity fusion
#### A. Process noise parameters
1. $gpsHorizPosNoise$ : GPS horizontal position measurement noise [m] <br>
&emsp; Note: This sets the GPS horizontal position observation noise. Increasing it reduces the weighting of GPS horizontal position measurements. <br>
</br>
2. $gpsHorizVelNoise$ : GPS horizontal velocity measurement noise [m/s] <br>
&emsp; Note: This sets a lower limit on the speed accuracy reported by the GPS receiver that is used to set horizontal velocity observation noise. If the model of receiver used does not provide a speed accurcy estimate, then the parameter value will be used. Increasing it reduces the weighting of the GPS horizontal velocity measurements. <br>
</br>
3. $posDownObsNoise$ : Observation noise variance on the vertical position used by the state and covariance update step [m^2] <br>
    a. $gpsHgtAccuracy$ : Estimated height accuracy returned by the GPS receiver [m] <br>
    </br>
    b. $rngNoise$ : Range finder noise [m] $(AP \; value: 0.5)$ <br>
    &emsp; Note: This is the RMS value of noise in the range finder measurement. Increasing it reduces the weighting on this measurement. <br>
    </br>
    c. $baroAltNoise$ : Baro height measurement noise [m] <br>
    &emsp; Note: This is the RMS value of noise in the altitude measurement. Increasing it reduces the weighting of the baro measurement and will make the filter respond more slowly to baro measurement errors, but will make it more sensitive to GPS and accelerometer errors. <br>
</br>
4. $gpsVertVelNoise$ : GPS vertical velocity measurement noise [m/s] <br>
&emsp; Note: This sets a lower limit on the speed accuracy reported by the GPS receiver that is used to set vertical velocity observation noise. If the model of receiver used does not provide a speed accurcy estimate, then the parameter value will be used. Increasing it reduces the weighting of the GPS vertical velocity measurements.

#### B. Measurement uncertainty parameters
5. $gpsPosAccuracy$ : Estimated position accuracy returned by the GPS receiver [m] $(AP \; value: 0)$ <br>
</br>
6. $gpsSpdAccuracy$ : Estimated speed accuracy returned by the GPS receiver [m/s] $(AP \; value: 0)$ <br>
</br>
7. $gpsHgtAccuracy$ : estimated height accuracy returned by the GPS receiver [m] $(AP \; value: 0)$ <br>

#### C. Innovation consistency check parameters
8. $gpsPosInnovGate$ : Percentage number of standard deviations applied to GPS position innovation consistency check [%] <br>
&emsp; Note: This sets the percentage number of standard deviations applied to the GPS position measurement innovation consistency check. Decreasing it makes it more likely that good measurements will be rejected. Increasing it makes it more likely that bad measurements will be accepted. <br>
</br>
9. $hgtInnovGate$ : Percentage number of standard deviations applied to height innovation consistency check [%] <br>
&emsp; Note: This sets the percentage number of standard deviations applied to the height measurement innovation consistency check. Decreasing it makes it more likely that good measurements will be rejected. Increasing it makes it more likely that bad measurements will be accepted. <br>
</br>
10. $gpsVelInnovGate$ : Percentage number of standard deviations applied to GPS velocity innovation consistency check [%] <br>
&emsp; Note: This sets the percentage number of standard deviations applied to the GPS velocity measurement innovation consistency check. Decreasing it makes it more likely that good measurements willbe rejected. Increasing it makes it more likely that bad measurements will be accepted.

## System model

The system model is represented in the form of the state extrapolation equation: <br>
<p style="text-align: center;"> $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$ <br> </p>
</br>
where, <br>
$X_{t|t-1}$ = Predicted system state at time t <br>
$F$ = State transition matrix <br>
$X_{t-1|t-1}$ = System state at time t-1 <br>
$G$ = Control matrix <br>
$U_{t-1|t-1}$ = Control vector at time t-1

In [37]:
# XT_predict_compact = Predicted system state at time t
# F = State transition matrix
# XT_previous = System state at time t-1
# G = Control matrix
# U = Control vector at time t-1

roll_t1, roll_rate_t1, roll_bias_t1, pitch_t1, pitch_rate_t1, pitch_bias_t1, roll_t, roll_rate_t, roll_acc_t, roll_bias_t, pitch_t, pitch_rate_t, pitch_acc_t, pitch_bias_t, delt = sp.symbols('\phi_{t|t-1}, \dot{\phi}_{t|t-1}, \phi_{b|t|t-1}, \\theta_{t|t-1}, \dot{\\theta}_{t|t-1}, \\theta_{b|t|t-1}, \phi_{t-1}, \dot{\phi}_{t-1}, \ddot{\phi}_{t-1}, \phi_{b|t-1}, \\theta_{t-1}, \dot{\\theta}_{t-1}, \ddot{\\theta}_{t-1}, \\theta_{b|t-1}, \Delta{t}')

Xt_predict_compact = sp.Matrix([[roll_t1],[roll_rate_t1],[roll_bias_t1],[pitch_t1],[pitch_rate_t1],[pitch_bias_t1]])
F = sp.Matrix([[1,delt,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,delt,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])
Xt_previous = sp.Matrix([[roll_t],[roll_rate_t],[roll_bias_t],[pitch_t],[pitch_rate_t],[pitch_bias_t]])
G = sp.Matrix([[delt**2/2,0],[delt,0],[0,0],[0,delt**2/2],[0,delt],[0,0]])
Ut_previous = sp.Matrix([[roll_acc_t],[pitch_acc_t]])

display(Math(r'X_{{t|t-1}} = {}, F = {}, X_{{t-1|t-1}} = {}, G = {}, U_{{t-1|t-1}} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

Arranging the matrices in the form of the state extrapolation equation: $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$

In [38]:
display(Math(r'{} = {}{} + {}{}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

## Predicted system state $X$ (at time t-1, for time t)

In [39]:
Xt_predict = F*Xt_previous+G*Ut_previous

display(Math(r'X_{{t|t-1}} = {} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(Xt_predict))))

<IPython.core.display.Math object>

## Process noise

In [43]:
# Q = Process noise covariance matrix
# q = Process noise standard deviation (sigma) of each state variable

Q_roll, Q_roll_rate, Q_roll_bias, Q_pitch, Q_pitch_rate, Q_pitch_bias = sp.symbols('q_{\phi}^2, q_{\dot{\phi}}^2, q_{{\phi}|b}^2, q_{\\theta}^2, q_{\dot{\\theta}}^2, q_{{\\theta}|b}^2')

Q = sp.Matrix([[Q_roll,0,0,0,0,0],[0,Q_roll_rate,0,0,0,0],[0,0,Q_roll_bias,0,0,0],[0,0,0,Q_pitch,0,0],[0,0,0,0,Q_pitch_rate,0],[0,0,0,0,0,Q_pitch_bias]])

display(Math(r'Q = {}'.format(sp.latex(Q))))

<IPython.core.display.Math object>

The process noise covariance matrix and its variables in the AP code are as follows:

In [50]:
Q_roll_AP, Q_roll_rate_AP, Q_roll_bias_AP, Q_pitch_AP, Q_pitch_rate_AP, Q_pitch_bias_AP = sp.symbols('{(gyrNoise*\Delta{t})}^2, {(gyrNoise)}^2, {(gyroBiasProcessNoise*(\Delta{t})^2)}^2, {(gyrNoise*\Delta{t})}^2, {(gyrNoise)}^2, {(gyroBiasProcessNoise*(\Delta{t})^2)}^2')

Q_AP = sp.Matrix([[Q_roll_AP,0,0,0,0,0],[0,Q_roll_rate_AP,0,0,0,0],[0,0,Q_roll_bias_AP,0,0,0],[0,0,0,Q_pitch_AP,0,0],[0,0,0,0,Q_pitch_rate_AP,0],[0,0,0,0,0,Q_pitch_bias_AP]])

display(Math(r'Q_{{AP}} = {}'.format(sp.latex(Q_AP))))

<IPython.core.display.Math object>

## Measurement uncertainty

In [106]:
# R = Measurement uncertainty covariance matrix
# r = Measurement uncertainty standard deviation (sigma) of each sensor that can measure a system state variable

R_pos_x, R_vel_x, R_pos_y, R_vel_y, R_pos_z, R_vel_z = sp.symbols('r_{x}^2, r_{\dot{x}}^2, r_{y}^2, r_{\dot{y}}^2, r_{z}^2, r_{\dot{z}}^2')

R = sp.Matrix([[R_pos_x,0,0,0,0,0],[0,R_vel_x,0,0,0,0],[0,0,R_pos_y,0,0,0],[0,0,0,R_vel_y,0,0],[0,0,0,0,R_pos_z,0],[0,0,0,0,0,R_vel_z]])

display(Math(r'R = {}'.format(sp.latex(R))))

<IPython.core.display.Math object>

The measurement uncertainty covariance matrix and its variables in the AP code are as follows:

In [113]:
R_pos_x_AP, R_vel_x_AP, R_pos_y_AP, R_vel_y_AP, R_pos_z_AP, R_vel_z_AP = sp.symbols('{(gpsPosAccuracy)}^2, {(gpsSpdAccuracy)}^2, {(gpsPosAccuracy)}^2, {(gpsSpdAccuracy)}^2, {(gpsHgtAccuracy)}^2, {(gpsSpdAccuracy)}^2')

R_AP = sp.Matrix([[R_pos_x_AP,0,0,0,0,0],[0,R_vel_x_AP,0,0,0,0],[0,0,R_pos_y_AP,0,0,0],[0,0,0,R_vel_y_AP,0,0],[0,0,0,0,R_pos_z_AP,0],[0,0,0,0,0,R_vel_z_AP]])

display(Math(r'R_{{AP}} = {}'.format(sp.latex(R_AP))))

<IPython.core.display.Math object>

## Initialized estimation uncertainty covariance matrix $P$ (at time t-1, for time t-1)

In [101]:
# Pt_previous = Estimated estimation uncertainty covariance matrix of the system state variables at time t-1

P_x, P_xdot, P_y, P_ydot, P_z, P_zdot = sp.symbols('P_x|t-1|t-1, P_{\dot{x}|t-1|t-1}, P_y|t-1|t-1, P_{\dot{y}|t-1|t-1}, P_z|t-1|t-1, P_{\dot{z}|t-1|t-1}')

Pt_previous = sp.Matrix([[P_x,0,0,0,0,0],[0,P_xdot,0,0,0,0],[0,0,P_y,0,0,0],[0,0,0,P_ydot,0,0,],[0,0,0,0,P_z,0],[0,0,0,0,0,P_zdot]])

display(Math(r'P_{{t-1|t-1}} = {}'.format(sp.latex(Pt_previous))))

<IPython.core.display.Math object>

## Predicted estimation uncertainty covariance matrix $P$ (at time t-1, for time t)

In [102]:
# Pt_predict = Predicted estimation uncertainty covariance matrix of the system state variables at time t

Pt_predict = (F*Pt_previous*F.T + Q)

display(Math(r'P_{{t|t-1}} = {}'.format(sp.latex(Pt_predict))))

<IPython.core.display.Math object>

## Updated measurement vector (at time t, for time t)

In [103]:
# Zt_update = Measurement vector at time t containing the sensor measurements
# H = Observation matrix to map the measurement vector to the system state variables

pos_xt, vel_xt, pos_yt, vel_yt, pos_zt, vel_zt = sp.symbols('z_{x|t}, z_{\dot{x}|t}, z_{y|t}, z_{\dot{y}|t}, z_{z|t}, z_{\dot{z}|t}')

Zt_update = sp.Matrix([[pos_xt],[vel_xt],[pos_yt],[vel_yt],[pos_zt],[vel_zt]])
H = sp.Matrix([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])

display(Math(r'Z_{{t|t}} = {}, H = {}'.format(sp.latex(Zt_update), sp.latex(H))))

<IPython.core.display.Math object>

The measurement vector and its variables in the AP code are as follows:

In [115]:
pos_xt_AP, vel_xt_AP, pos_yt_AP, vel_yt_AP, pos_zt_AP, vel_zt_AP = sp.symbols('posxy.x, gpsDataDelayed.vel.x, posxy.y, gpsDataDelayed.vel.y, gpsDataDelayed.hgt, gpsDataDelayed.vel.z')

Zt_update_AP = sp.Matrix([[pos_xt_AP],[vel_xt_AP],[pos_yt_AP],[vel_yt_AP],[pos_zt_AP],[vel_zt_AP]])
display(Math(r'Z_{{t|t|AP}} = {}'.format(sp.latex(Zt_update_AP))))

<IPython.core.display.Math object>

#### Note on $z_{z|t}$:
If height measurement source = GPS <br>
&emsp; $z_{z|t} = gpsDataDelayed.hgt$ <br>
Else <br>
If height measurement source = Rangefinder <br>
&emsp; $z_{z|t} = rangeDataDelayed.hgt$ <br>
Else <br>
If height measurement source = Barometer <br>
&emsp; $z_{z|t} = baroDataDelayed.hgt$

## Updated innovation matrix (at time t, for time t)

In [105]:
# Yt_update = Updated innovation matrix at time t

Yt_update = H*Xt_predict_compact - Zt_update

display(Math(r'Y_{{t|t}} = {}'.format(sp.latex(Yt_update))))

<IPython.core.display.Math object>

## Updated innovation covariance matrix (at time t, for time t)

In [108]:
# St_update = Updated innovation covariance matrix at time t

St_update = H*Pt_predict*H.T + R

display(Math(r'S_{{t|t}} = {}'.format(sp.latex(St_update))))

<IPython.core.display.Math object>

## Innovation consistency check on horizontal (x-y) positions

In [109]:
Yt_11, Yt_31, maxPosInnovXY = sp.symbols('Y_{t|t|11}, Y_{t|t|31}, maxPosInnovXY')

display(Math(r'maxPosInnovXY = max(0.01*gpsPosInnovGate, 1)^2 * (S_{{t|t|11}} + S_{{t|t|33}})'))

posTestRatio = ((Yt_11)**2 + (Yt_31)**2) / maxPosInnovXY

display(Math(r'posTestRatio = {}'.format(sp.latex(posTestRatio))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Condition for successful fusion of horizontal (x-y) positions data:
If (posTestRatio < 1) <br>
&emsp; fusePosData = True; <br>
Else <br>
&emsp; fusePosData = False;

## Innovation consistency check on vertical (z) position

In [110]:
Yt_51, maxPosInnovZ = sp.symbols('Y_{t|t|51}, maxPosInnovZ')

display(Math(r'maxPosInnovZ = max(0.01*hgtInnovGate, 1)^2 * S_{{t|t|55}}'))

hgtTestRatio = (Yt_51)**2 / maxPosInnovZ

display(Math(r'hgtTestRatio = {}'.format(sp.latex(hgtTestRatio))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Condition for successful fusion of vertical (z) position data:
If (PV_AidingMode == AID_NONE && onGround) <br>
&emsp; maxTestRatio = 3; <br>
Else <br>
&emsp; maxTestRatio = 1; <br>
</br>
If (hgtTestRatio < maxTestRatio) <br>
&emsp; fuseHgtData = True; <br>
Else <br>
&emsp; fuseHgtData = False;

## Innovation consistency check on (x-y-z) velocities

In [111]:
Yt_21, Yt_41, Yt_61, maxVelInnovXYZ = sp.symbols('Y_{t|t|21}, Y_{t|t|41}, Y_{t|t|61}, maxVelInnovXYZ')

display(Math(r'maxVelInnovXYZ = max(0.01*gpsVelInnovGate, 1)^2 * (S_{{t|t|22}} + S_{{t|t|44}} + S_{{t|t|66}})'))

velTestRatio = ((Yt_21)**2 + (Yt_41)**2 + (Yt_61)**2) / maxVelInnovXYZ

display(Math(r'velTestRatio = {}'.format(sp.latex(velTestRatio))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### Condition for successful fusion of (x-y-z) velocities data:
If (velTestRatio < 1) <br>
&emsp; fuseVelData = True; <br>
Else <br>
&emsp; fuseVelData = False;

In [19]:
Num = sp.adjoint(St_update).simplify()

In [20]:
Den = sp.det(St_update).simplify()

In [21]:
St_inv = (Num/Den).simplify()

display(Math(r'S_{{t|t}}^{{-1}} = {}'.format(sp.latex(St_inv))))

<IPython.core.display.Math object>

## Updated Kalman gain $K$ (at time t, for time t)

In [24]:
# Kt_update = Updated Kalman gain at time t

Kt_update = (Pt_predict*H.T*St_inv).simplify()

display(Math(r'K_{{t|t}} = {}'.format(sp.latex(Kt_update))))

KeyboardInterrupt: 

## Estimated system state $X$ (at time t, for time t)

In [66]:
# Xt_estimate = Estimated system state at time t

Xt_estimate = (Xt_predict_compact + Kt_update*(Yt_update))

display(Math(r'X_{{t|t}} = {}'.format(sp.latex(Xt_estimate))))

<IPython.core.display.Math object>

## Estimated estimation uncertainty covariance matrix $P$ (at time t, for time t)

In [67]:
# Pt_estimate = Estimated estimation uncertainty matrix of the system state variables at time t

I = sp.Matrix([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])

Pt_estimate = ((I - Kt_update*H)*Pt_predict)

display(Math(r'P_{{t|t}} = {}'.format(sp.latex(Pt_estimate))))

<IPython.core.display.Math object>